In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
from keras import backend as K
K.set_image_dim_ordering('th') 

import numpy as np
import pandas as pd
import cv2
import zarr
import glob
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input
from keras.optimizers import Adam, SGD, RMSprop, Nadam

from keras.layers.convolutional import Convolution3D, MaxPooling3D, UpSampling3D
from keras.layers.advanced_activations import PReLU
from keras.layers import BatchNormalization, GlobalAveragePooling3D, GlobalMaxPooling3D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.core import SpatialDropout3D
from keras.utils.np_utils import to_categorical


from preds3d_utils import *
from preds3d_models_exp import *

### 1398 stage1 original examples

In [ ]:
def classifier(kernel_size, pool_size):
    model = Sequential()

    model.add(Convolution3D(16, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',
                            input_shape=(1, 136, 168, 168)))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(32, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Convolution3D(64, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=5e-5), metrics = ['accuracy'])
    return model

In [ ]:
def classifier_exp(width, ks1, ks2, ks3):
    
    inputs = Input(shape=(1, 136, 168, 168))
    conv1 = Convolution3D(width, ks1[0], ks1[1], ks1[2], activation = 'relu', border_mode='same')(inputs)
    conv1 = Convolution3D(width, ks1[0], ks1[1], ks1[2], activation = 'relu', border_mode='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2), border_mode='same')(conv1)
    
    conv2 = Convolution3D(width, ks2[0], ks2[1], ks2[2], activation = 'relu', border_mode='same')(pool1)
    conv2 = Convolution3D(width, ks2[0], ks2[1], ks2[2], activation = 'relu', border_mode='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2), border_mode='same')(conv2)

    conv3 = Convolution3D(width, ks3[0], ks3[1], ks3[2], activation = 'relu', border_mode='same')(pool2)
    conv3 = Convolution3D(width, ks3[0], ks3[1], ks3[2], activation = 'relu', border_mode='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2), border_mode='same')(conv3)
    
    output = Flatten(name='flatten')(pool3)
    output = Dropout(0.2)(output)
    output = Dense(128)(output)
    output = PReLU()(output)
    output = Dropout(0.2)(output)
    output = Dense(128)(output)
    output = PReLU()(output)
    output = Dropout(0.3)(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    
    model3d = Model(inputs, output)
    model3d.compile(loss='categorical_crossentropy', optimizer = Adam(lr=5e-5), metrics = ['accuracy'])
    return model3d

In [ ]:
def cnn3d(width, ks1, ks2, ks3):
    
    inputs = Input(shape=(1, 136, 168, 168))
    conv1 = Convolution3D(width, ks1[0], ks1[1], ks1[2], activation = 'relu', border_mode='same')(inputs)
    conv1 = Convolution3D(width, ks1[0], ks1[1], ks1[2], activation = 'relu', border_mode='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2), border_mode='same')(conv1)
    
    conv2 = Convolution3D(width, ks2[0], ks2[1], ks2[2], activation = 'relu', border_mode='same')(pool1)
    conv2 = Convolution3D(width, ks2[0], ks2[1], ks2[2], activation = 'relu', border_mode='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2), border_mode='same')(conv2)

    conv3 = Convolution3D(width, ks3[0], ks3[1], ks3[2], activation = 'relu', border_mode='same')(pool2)
    conv3 = Convolution3D(width, ks3[0], ks3[1], ks3[2], activation = 'relu', border_mode='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(8, 8, 8), border_mode='same')(conv3)
    
    output = Flatten(name='flatten')(pool3)
    output = Dropout(0.2)(output)
    output = Dense(128)(output)
    output = PReLU()(output)
    output = Dropout(0.2)(output)
    output = Dense(128)(output)
    output = PReLU()(output)
    output = Dropout(0.3)(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    
    model3d = Model(inputs, output)
    model3d.compile(loss='categorical_crossentropy', optimizer = Adam(lr=5e-5), metrics = ['accuracy'])
    return model3d

In [ ]:
start_train = 0
end_train = 1398
start_val = 1400
end_val = 1595

epochs = 20

width = 16
ks1 = (2, 2, 2)
ks2 = ks3 = ks1

In [ ]:
cnn3d_genfit('DSB_class3d_preds3dexp', classifier(ks1, ks2), epochs, start_train, end_train, start_val, end_val, 
             end_train - start_train,
             end_val - start_val)

In [ ]:
cnn3d_genfit('DSB_class3d_preds3dexp', cnn3d(width, ks1, ks2, ks3), epochs, start_train, end_train, start_val, end_val, 
             end_train - start_train,
             end_val - start_val)

In [ ]:
cnn3d_genfit('DSB_class3d_preds3globalavg_1450pats', preds3d_globalavg_highlr(width), epochs, start_train, end_train, start_val, end_val, 
             end_train - start_train,
             end_val - start_val)